In [6]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict
from shapely.geometry import box
import torchvision
import torch
import math
from datetime import datetime

df = pd.read_csv('FINAL.csv')

def parse_location(loc):
    return np.array(list(map(float, loc.split(':'))))

def timestamp_to_seconds(ts):
    hours, minutes, seconds = ts.split('_')
    total_seconds = int(hours) * 3600 + int(minutes) * 60 + float(seconds)
    return total_seconds

def find_second_to_last(arr, target):
    indices = [i for i, value in enumerate(arr) if value == target]
    if len(indices) >= 2:
        return indices[-2]
    return None

def calculate_overlap(box1, box2):
    x1_1, y1_1, x2_1, y2_1 = box1[0], box1[1], box1[0] + box1[2], box1[1] + box1[3]
    x1_2, y1_2, x2_2, y2_2 = box2[0], box2[1], box2[0] + box2[2], box2[1] + box2[3]

    x1_inter = max(x1_1, x1_2)
    y1_inter = max(y1_1, y1_2)
    x2_inter = min(x2_1, x2_2)
    y2_inter = min(y2_1, y2_2)

    inter_width = max(0, x2_inter - x1_inter)
    inter_height = max(0, y2_inter - y1_inter)
    overlap_area = inter_width * inter_height

    return overlap_area

def euclidean_distance_cxcywh(box1, box2):
    cx1, cy1, _, _ = box1
    cx2, cy2, _, _ = box2
    distance = math.sqrt((cx1 - cx2) ** 2 + (cy1 - cy2) ** 2)
    return distance

df['timestamp_seconds'] = df['Timestamp'].apply(timestamp_to_seconds)
df['location_coords'] = df['Location'].apply(parse_location)
df = df.sort_values(by=['VideoFile', 'timestamp_seconds']).reset_index(drop=True)

final_ids = {}
unique_ids = {}
temp_ids = {}
result = []
previous_time = None

for index, row in df.iterrows():
    pred = row['Prediction']
    ts = row['timestamp_seconds']
    loc = row['location_coords']
    vf = row['VideoFile']
    if ts != previous_time:
        final_ids.update(temp_ids)
        temp_ids={}
    
    # Predictions with the same name within 2 seconds
    candidates = [
    (uid, info) for uid, info in final_ids.items() 
        if info['Prediction'] == pred and abs(info['timestamp_seconds'] - ts) <= 2 and info['VideoFile'] == vf
    ]
    if candidates:
        distances = [
            (
                uid, 
                np.linalg.norm(info['location_coords'] - loc) + 10 * (abs(info['timestamp_seconds'] - ts) // 0.25)
            )
            for uid, info in candidates
        ]
        closest_uid, closest_dist = min(distances, key=lambda x: x[1]) #Find the closest prediction
        if closest_dist < 5000:  # Within a certain distance
            row_numbers = [index for index, info in enumerate(unique_ids.values()) if info['timestamp_seconds'] == ts and info['unique_id'] == closest_uid] # is there another row with the same timestamp and prediction
            if row_numbers:
                timesb = list(unique_ids.values())[row_numbers[0]]['timestamp_seconds']
                locb = list(unique_ids.values())[row_numbers[0]]['location_coords']
                predb = list(unique_ids.values())[row_numbers[0]]['Prediction']
                
                indx = find_second_to_last(result, closest_uid)
                if indx:
                    tarlocx = df.loc[indx, 'location_coords']
                    distance_a = euclidean_distance_cxcywh(loc, tarlocx)
                    distance_b = euclidean_distance_cxcywh(locb, tarlocx)

                    if distance_a < distance_b:
                        unique_id = closest_uid
                        locb_indx = len(result) - 1 - result[::-1].index(closest_uid)
                        
                        distances = [row for row in distances if row[0] != closest_uid] # find next best
                        next_best = False
                        if distances and len(distances) < 3:
                            closest_uid, closest_dist = min(distances, key=lambda x: x[1])
                            row_numbers = [index for index, info in enumerate(unique_ids.values()) if info['timestamp_seconds'] == ts and info['unique_id'] == closest_uid]
                            next_best = True
                        if next_best == True and not row_numbers:
                            new_id = closest_uid
                            result[locb_indx] = new_id
                            unique_ids[new_id] = {'timestamp_seconds': timesb, 'location_coords': locb, 'Prediction': predb, 'unique_id': new_id, 'VideoFile' : vf}
                            final_ids[new_id] = {'timestamp_seconds': timesb, 'location_coords': locb, 'Prediction': predb, 'unique_id': new_id, 'VideoFile' : vf}
                        else:
                            new_id = f"{pred}_{len(set(result)) + 1:04d}"
                            result[locb_indx] = new_id
                            unique_ids[new_id] = {'timestamp_seconds': timesb, 'location_coords': locb, 'Prediction': predb, 'unique_id': new_id, 'VideoFile' : vf}
                            final_ids[new_id] = {'timestamp_seconds': timesb, 'location_coords': locb, 'Prediction': predb, 'unique_id': new_id, 'VideoFile' : vf}

                    
                    else:
                        distances = [row for row in distances if row[0] != closest_uid] # find next best
                        next_best = False
                        if distances and len(distances) < 3:
                            closest_uid, closest_dist = min(distances, key=lambda x: x[1])
                            row_numbers = [index for index, info in enumerate(unique_ids.values()) if info['timestamp_seconds'] == ts and info['unique_id'] == closest_uid]
                            next_best = True
                        if next_best == True and not row_numbers:
                            unique_id = closest_uid
                        else:
                            unique_id = f"{pred}_{len(set(result)) + 1:04d}" 
                else: # This means the previous occurance was the first so just make a new one
                    unique_id = f"{pred}_{len(set(result)) + 1:04d}"
            else: #This one means that there are no other rows in unique_ids with the same timestamp or unique_id
                unique_id = closest_uid
        else: # This one means that the distance is too far (unlikley)
            unique_id = f"{pred}_{len(set(result)) + 1:04d}" 
    else: #There are no previous prediction with the same name and in the last 2 seconds
        unique_id = f"{pred}_{len(set(result)) + 1:04d}"
        
    temp_ids[unique_id] = {'timestamp_seconds': ts, 'location_coords': loc, 'Prediction': pred, 'unique_id': unique_id, 'VideoFile' : vf}
    unique_ids[unique_id] = {'timestamp_seconds': ts, 'location_coords': loc, 'Prediction': pred, 'unique_id': unique_id, 'VideoFile' : vf}
    previous_time = ts
    result.append(unique_id)

df['unique_id'] = result
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format='%H_%M_%S.%f')
df['Time'] = df['Timestamp'].dt.strftime('%H:%M:%S.%f').str[:-3]

def average_location(group):
    val = group.mean()
    return val

df[['x1', 'y1', 'x2', 'y2']] = pd.DataFrame(df['location_coords'].tolist(), index=df.index)

result = df.groupby('unique_id').agg(
    Student=('Student', 'first'),
    Device=('Device', 'first'),
    DateTime=('DateTime', 'first'),
    VideoFile=('VideoFile', 'first'),
    start_time=('Time', 'min'),
    end_time=('Time', 'max'),
    prediction=('Prediction', 'first'),
    avg_confidence=('Confidence', lambda x: round(x.mean(), 2)),
    frame_count=('Timestamp', 'count'),
    avg_location_x1=('x1', 'median'),
    avg_location_y1=('y1', 'median'),
    avg_location_x2=('x2', 'median'),
    avg_location_y2=('y2', 'median')
).reset_index()

result['avg_location'] = list(zip(result['avg_location_x1'], result['avg_location_y1'], result['avg_location_x2'], result['avg_location_y2']))
result = result.drop(columns=['avg_location_x1', 'avg_location_y1', 'avg_location_x2', 'avg_location_y2'])
result = result[result['frame_count'] > 1]
result = result[result['avg_confidence'] > 0.71]
result = result.drop(columns=['unique_id'])

def filter(df):
    df = df.sort_values(by=['VideoFile', 'start_time']).reset_index(drop=True)
    VideoFile = df['VideoFile'].iloc[0]
    End_Time = df['end_time'].iloc[0]
    filtered_rows = []
    temp_rows = []
    New = True

    for i in range(len(df)):
        if df['VideoFile'].iloc[i] != VideoFile:
            VideoFile = df['VideoFile'].iloc[i]
            End_Time = df['end_time'].iloc[i]
            New = True
        if temp_rows:
            End_Time = max(df['end_time'].iloc[temp_rows])
        if New or df['start_time'].iloc[i] > End_Time:
            if temp_rows:
                filtered_rows.extend(temp_rows)
                temp_rows = []
            End_Time = df['end_time'].iloc[i]
            temp_rows.append(i)
        else:
            Add = True
            for j in temp_rows:
                if df['start_time'].iloc[i] < df['end_time'].iloc[j]:
                    l_i = torchvision.ops.box_convert(torch.tensor(df['avg_location'].iloc[i]), in_fmt = "cxcywh", out_fmt = "xywh").numpy()
                    l_j = torchvision.ops.box_convert(torch.tensor(df['avg_location'].iloc[j]), in_fmt = "cxcywh", out_fmt = "xywh").numpy()
                    if calculate_overlap(l_i, l_j) > 100:
                        base_i = df.loc[i, 'prediction'].split('-', 1)[0]
                        base_j = df.loc[j, 'prediction'].split('-', 1)[0]
                        if base_i == base_j:
                            df.loc[j, 'start_time'] = min(df.loc[i, 'start_time'], df.loc[j, 'start_time'])
                            df.loc[j, 'end_time'] = max(df.loc[i, 'end_time'], df.loc[j, 'end_time'])
                            df.loc[j, 'avg_confidence'] = max(df.loc[i, 'avg_confidence'], df.loc[j, 'avg_confidence'])
                            df.loc[j, 'frame_count'] = df.loc[i, 'frame_count'] + df.loc[j, 'frame_count']
                            df.loc[j, 'prediction'] = base_i
                            Add = False
                        else:
                            if df['avg_confidence'].iloc[i] > df['avg_confidence'].iloc[j]:
                                temp_rows.remove(j)
                                temp_rows.append(i)
                                Add = False
                            else:
                                Add = False
            if Add == True:
                temp_rows.append(i)
        New=False
        if i == len(df)-1:
            filtered_rows.extend(temp_rows)
    return df.iloc[filtered_rows]

def split_datetime(s):
    parts = s.split("_")
    date_str = f"{parts[0]} {parts[1]} {parts[2]}".strip()
    date = datetime.strptime(date_str, "%b %d %Y").strftime("%m/%d/%Y")
    time_str = f"{parts[3][:2]}:{parts[3][2:]} {parts[4]}"  # Time in 12-hour format (e.g., 1028 AM)
    time_24hr = datetime.strptime(time_str, "%I:%M %p").strftime("%H:%M")
    return pd.Series([date, time_24hr])

result = filter(result)
result = result[~((result['frame_count'] < 4) & (result['avg_confidence'] <= 0.80))] 
result = result.sort_values(by=['VideoFile', 'start_time']).reset_index(drop=True)
result[['Date', 'Time']] = result['DateTime'].apply(split_datetime)
result = result.drop(columns=['avg_location', 'DateTime', 'VideoFile'])
result.to_csv('final_processed_predictions_brands.csv', index=False)

In [5]:
import pandas as pd

def get_unique_rows(file_path, output_file):
    # Read the CSV file
    df = pd.read_csv(file_path)
    
    # Drop duplicate rows based on 'Student', 'Divide', and 'DateTime'
    unique_df = df.drop_duplicates(subset=['Student', 'Device', 'DateTime'])
    
    # Save the unique rows to a new CSV file
    unique_df.to_csv(output_file, index=False)
    
    return unique_df

if __name__ == "__main__":
    file_path = "FINAL.csv"  # Replace with your actual file path
    output_file = "output.csv"  # Replace with desired output file path
    unique_rows = get_unique_rows(file_path, output_file)
    print(f"Unique rows saved to {output_file}")



Unique rows saved to output.csv
